# 변수 설명 및 대회 전략
+ GitHub README에 다 적어놨습니다.
+ 중복데이터 처리를 어떻게 해야할지 고민을 하였습니다.
+ Feature Engineering보단 Catboost의 Categorical 데이터에 관한 하이퍼파라미터 설정에 포커스를 맞췄습니다.
+ [GitHub](https://github.com/ds-wook/PredictCreditCardDelinquency)

# 참고한 자료
+ [Hayo 님의 노트북](https://dacon.io/competitions/official/235713/codeshare/2519?page=1&dtype=recent)
+ [rollcake님의 노트북](https://dacon.io/competitions/official/235713/codeshare/2526?page=1&dtype=recent)
+ [datu님의 노트북](https://dacon.io/competitions/official/235713/codeshare/2515?page=2&dtype=recent)
+ [catboost 논문](https://arxiv.org/pdf/1706.09516.pdf)
+ [catboost 활용 논문](https://arxiv.org/abs/2104.07553)

In [1]:
!pip uninstall -y typing # this should avoid  AttributeError: type object 'Callable' has no attribute '_abc_registry'
!pip install  "git+https://github.com/dreamquark-ai/tabnet.git@develop#egg=pytorch_tabnet" --upgrade

  Cloning https://github.com/dreamquark-ai/tabnet.git (to revision develop) to /tmp/pip-install-2ke2ioc2/pytorch-tabnet_05b3f8a626bb4a6994d213e123b81bcb
  Running command git clone -q https://github.com/dreamquark-ai/tabnet.git /tmp/pip-install-2ke2ioc2/pytorch-tabnet_05b3f8a626bb4a6994d213e123b81bcb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pytorch-tabnet: filename=pytorch_tabnet-3.1.1-py3-none-any.whl size=39326 sha256=dbf370437df193bcfe8cb3ee932ec034b2624d8b3cea2c676b2862d1c401b1dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-xa0gkqiu/wheels/a6/8e/aa/6f5ef6a2e389c8b5f7ea1c74bbb03ece8773b03c2b8955c334
Successfully built pytorch-tabnet


In [1]:
from typing import Dict, Tuple, Union, List

import numpy as np
import pandas as pd
import torch

from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\JangSeongHyun\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-cc5861bb394b>", line 8, in <module>
    from lightgbm import LGBMClassifier
  File "C:\Users\JangSeongHyun\anaconda3\lib\site-packages\lightgbm\__init__.py", line 8, in <module>
    from .basic import Booster, Dataset
  File "C:\Users\JangSeongHyun\anaconda3\lib\site-packages\lightgbm\basic.py", line 14, in <module>
    from .compat import (PANDAS_INSTALLED, DataFrame, Series, is_dtype_sparse,
  File "C:\Users\JangSeongHyun\anaconda3\lib\site-packages\lightgbm\compat.py", line 114, in <module>
    from sklearn.base import BaseEstimator
  File "C:\Users\JangSeongHyun\anaconda3\lib\site-packages\sklearn\__init__.py", line 82, in <module>
    from .base import clone
  File "C:\Users\JangSeongHyun\anaconda3\lib\site-packages\sklearn\base.py", line 17, in <module>

TypeError: object of type 'NoneType' has no len()

# Load Dataset
+ catboost가 이 대회의 핵심 알고리즘이라고 생각하여 categorical한 변수를 만드는게 중요하다고 생각하여 따로 함수를 만듬
+ 다른 알고리즘들은 categorical한 변수보다 numeric한 변수 처리에 초점을 맞춤

In [3]:
def category_income(data: pd.DataFrame) -> pd.DataFrame:
    data["income_total"] = data["income_total"] / 10000
    conditions = [
        (data["income_total"].le(18)),
        (data["income_total"].gt(18) & data["income_total"].le(33)),
        (data["income_total"].gt(33) & data["income_total"].le(49)),
        (data["income_total"].gt(49) & data["income_total"].le(64)),
        (data["income_total"].gt(64) & data["income_total"].le(80)),
        (data["income_total"].gt(80) & data["income_total"].le(95)),
        (data["income_total"].gt(95) & data["income_total"].le(111)),
        (data["income_total"].gt(111) & data["income_total"].le(126)),
        (data["income_total"].gt(126) & data["income_total"].le(142)),
        (data["income_total"].gt(142)),
    ]
    choices = [i for i in range(10)]

    data["income_total"] = np.select(conditions, choices)
    return data


def load_dataset() -> Tuple[pd.DataFrame, pd.DataFrame]:
    path = "../input/predictcreditcarddelinquency/"
    train = pd.read_csv(path + "train.csv")
    train = train.drop(["index"], axis=1)
    train.fillna("NAN", inplace=True)

    test = pd.read_csv(path + "test.csv")
    test = test.drop(["index"], axis=1)
    test.fillna("NAN", inplace=True)

    # absolute
    train["DAYS_EMPLOYED"] = train["DAYS_EMPLOYED"].map(lambda x: 0 if x > 0 else x)
    train["DAYS_EMPLOYED"] = np.abs(train["DAYS_EMPLOYED"])
    test["DAYS_EMPLOYED"] = test["DAYS_EMPLOYED"].map(lambda x: 0 if x > 0 else x)
    test["DAYS_EMPLOYED"] = np.abs(test["DAYS_EMPLOYED"])
    train["DAYS_BIRTH"] = np.abs(train["DAYS_BIRTH"])
    test["DAYS_BIRTH"] = np.abs(test["DAYS_BIRTH"])
    train["begin_month"] = np.abs(train["begin_month"]).astype(int)
    test["begin_month"] = np.abs(test["begin_month"]).astype(int)

    # DAYS_BIRTH
    train["DAYS_BIRTH_month"] = np.floor(train["DAYS_BIRTH"] / 30) - (
        (np.floor(train["DAYS_BIRTH"] / 30) / 12).astype(int) * 12
    )
    train["DAYS_BIRTH_month"] = train["DAYS_BIRTH_month"].astype(int)
    train["DAYS_BIRTH_week"] = np.floor(train["DAYS_BIRTH"] / 7) - (
        (np.floor(train["DAYS_BIRTH"] / 7) / 4).astype(int) * 4
    )
    train["DAYS_BIRTH_week"] = train["DAYS_BIRTH_week"].astype(int)
    test["DAYS_BIRTH_month"] = np.floor(test["DAYS_BIRTH"] / 30) - (
        (np.floor(test["DAYS_BIRTH"] / 30) / 12).astype(int) * 12
    )
    test["DAYS_BIRTH_month"] = test["DAYS_BIRTH_month"].astype(int)
    test["DAYS_BIRTH_week"] = np.floor(test["DAYS_BIRTH"] / 7) - (
        (np.floor(test["DAYS_BIRTH"] / 7) / 4).astype(int) * 4
    )
    test["DAYS_BIRTH_week"] = test["DAYS_BIRTH_week"].astype(int)

    # Age
    train["Age"] = np.abs(train["DAYS_BIRTH"]) // 360
    test["Age"] = np.abs(test["DAYS_BIRTH"]) // 360

    # DAYS_EMPLOYED
    train["DAYS_EMPLOYED_month"] = np.floor(train["DAYS_EMPLOYED"] / 30) - (
        (np.floor(train["DAYS_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    train["DAYS_EMPLOYED_month"] = train["DAYS_EMPLOYED_month"].astype(int)
    train["DAYS_EMPLOYED_week"] = np.floor(train["DAYS_EMPLOYED"] / 7) - (
        (np.floor(train["DAYS_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    train["DAYS_EMPLOYED_week"] = train["DAYS_EMPLOYED_week"].astype(int)
    test["DAYS_EMPLOYED_month"] = np.floor(test["DAYS_EMPLOYED"] / 30) - (
        (np.floor(test["DAYS_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    test["DAYS_EMPLOYED_month"] = test["DAYS_EMPLOYED_month"].astype(int)
    test["DAYS_EMPLOYED_week"] = np.floor(test["DAYS_EMPLOYED"] / 7) - (
        (np.floor(test["DAYS_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    test["DAYS_EMPLOYED_week"] = test["DAYS_EMPLOYED_week"].astype(int)

    # EMPLOYED
    train["EMPLOYED"] = train["DAYS_EMPLOYED"] / 360
    test["EMPLOYED"] = test["DAYS_EMPLOYED"] / 360

    # before_EMPLOYED
    train["before_EMPLOYED"] = train["DAYS_BIRTH"] - train["DAYS_EMPLOYED"]
    train["before_EMPLOYED_month"] = np.floor(train["before_EMPLOYED"] / 30) - (
        (np.floor(train["before_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    train["before_EMPLOYED_month"] = train["before_EMPLOYED_month"].astype(int)
    train["before_EMPLOYED_week"] = np.floor(train["before_EMPLOYED"] / 7) - (
        (np.floor(train["before_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    train["before_EMPLOYED_week"] = train["before_EMPLOYED_week"].astype(int)
    test["before_EMPLOYED"] = test["DAYS_BIRTH"] - test["DAYS_EMPLOYED"]
    test["before_EMPLOYED_month"] = np.floor(test["before_EMPLOYED"] / 30) - (
        (np.floor(test["before_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    test["before_EMPLOYED_month"] = test["before_EMPLOYED_month"].astype(int)
    test["before_EMPLOYED_week"] = np.floor(test["before_EMPLOYED"] / 7) - (
        (np.floor(test["before_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    test["before_EMPLOYED_week"] = test["before_EMPLOYED_week"].astype(int)

    # gender_car_reality
    train["user_code"] = (
        train["gender"].astype(str)
        + "_"
        + train["car"].astype(str)
        + "_"
        + train["reality"].astype(str)
    )
    test["user_code"] = (
        test["gender"].astype(str)
        + "_"
        + test["car"].astype(str)
        + "_"
        + test["reality"].astype(str)
    )

    del_cols = [
        "gender",
        "car",
        "reality",
        "email",
        "child_num",
        "DAYS_BIRTH",
        "DAYS_EMPLOYED",
    ]
    train.drop(train.loc[train["family_size"] > 7, "family_size"].index, inplace=True)
    train.drop(del_cols, axis=1, inplace=True)
    test.drop(del_cols, axis=1, inplace=True)

    cat_cols = [
        "income_type",
        "edu_type",
        "family_type",
        "house_type",
        "occyp_type",
        "user_code",
    ]

    for col in cat_cols:
        label_encoder = LabelEncoder()
        label_encoder = label_encoder.fit(train[col])
        train[col] = label_encoder.transform(train[col])
        test[col] = label_encoder.transform(test[col])

    return train, test


def cat_load_dataset() -> Tuple[pd.DataFrame, pd.DataFrame]:
    path = "../input/predictcreditcarddelinquency/"
    train = pd.read_csv(path + "train.csv")
    train = train.drop(["index"], axis=1)
    train.fillna("NAN", inplace=True)

    test = pd.read_csv(path + "test.csv")
    test = test.drop(["index"], axis=1)
    test.fillna("NAN", inplace=True)

    # absolute
    train["DAYS_EMPLOYED"] = train["DAYS_EMPLOYED"].map(lambda x: 0 if x > 0 else x)
    train["DAYS_EMPLOYED"] = np.abs(train["DAYS_EMPLOYED"])
    test["DAYS_EMPLOYED"] = test["DAYS_EMPLOYED"].map(lambda x: 0 if x > 0 else x)
    test["DAYS_EMPLOYED"] = np.abs(test["DAYS_EMPLOYED"])
    train["DAYS_BIRTH"] = np.abs(train["DAYS_BIRTH"])
    test["DAYS_BIRTH"] = np.abs(test["DAYS_BIRTH"])
    train["begin_month"] = np.abs(train["begin_month"]).astype(int)
    test["begin_month"] = np.abs(test["begin_month"]).astype(int)

    # income_total
    train = category_income(train)
    test = category_income(test)

    # DAYS_BIRTH
    train["DAYS_BIRTH_month"] = np.floor(train["DAYS_BIRTH"] / 30) - (
        (np.floor(train["DAYS_BIRTH"] / 30) / 12).astype(int) * 12
    )
    train["DAYS_BIRTH_month"] = train["DAYS_BIRTH_month"].astype(int)
    train["DAYS_BIRTH_week"] = np.floor(train["DAYS_BIRTH"] / 7) - (
        (np.floor(train["DAYS_BIRTH"] / 7) / 4).astype(int) * 4
    )
    train["DAYS_BIRTH_week"] = train["DAYS_BIRTH_week"].astype(int)
    test["DAYS_BIRTH_month"] = np.floor(test["DAYS_BIRTH"] / 30) - (
        (np.floor(test["DAYS_BIRTH"] / 30) / 12).astype(int) * 12
    )
    test["DAYS_BIRTH_month"] = test["DAYS_BIRTH_month"].astype(int)
    test["DAYS_BIRTH_week"] = np.floor(test["DAYS_BIRTH"] / 7) - (
        (np.floor(test["DAYS_BIRTH"] / 7) / 4).astype(int) * 4
    )
    test["DAYS_BIRTH_week"] = test["DAYS_BIRTH_week"].astype(int)

    # Age
    train["Age"] = np.abs(train["DAYS_BIRTH"]) // 360
    test["Age"] = np.abs(test["DAYS_BIRTH"]) // 360

    # DAYS_EMPLOYED
    train["DAYS_EMPLOYED_month"] = np.floor(train["DAYS_EMPLOYED"] / 30) - (
        (np.floor(train["DAYS_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    train["DAYS_EMPLOYED_month"] = train["DAYS_EMPLOYED_month"].astype(int)
    train["DAYS_EMPLOYED_week"] = np.floor(train["DAYS_EMPLOYED"] / 7) - (
        (np.floor(train["DAYS_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    train["DAYS_EMPLOYED_week"] = train["DAYS_EMPLOYED_week"].astype(int)
    test["DAYS_EMPLOYED_month"] = np.floor(test["DAYS_EMPLOYED"] / 30) - (
        (np.floor(test["DAYS_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    test["DAYS_EMPLOYED_month"] = test["DAYS_EMPLOYED_month"].astype(int)
    test["DAYS_EMPLOYED_week"] = np.floor(test["DAYS_EMPLOYED"] / 7) - (
        (np.floor(test["DAYS_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    test["DAYS_EMPLOYED_week"] = test["DAYS_EMPLOYED_week"].astype(int)

    # EMPLOYED
    train["EMPLOYED"] = train["DAYS_EMPLOYED"] / 360
    test["EMPLOYED"] = test["DAYS_EMPLOYED"] / 360

    # before_EMPLOYED
    train["before_EMPLOYED"] = train["DAYS_BIRTH"] - train["DAYS_EMPLOYED"]
    train["before_EMPLOYED_month"] = np.floor(train["before_EMPLOYED"] / 30) - (
        (np.floor(train["before_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    train["before_EMPLOYED_month"] = train["before_EMPLOYED_month"].astype(int)
    train["before_EMPLOYED_week"] = np.floor(train["before_EMPLOYED"] / 7) - (
        (np.floor(train["before_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    train["before_EMPLOYED_week"] = train["before_EMPLOYED_week"].astype(int)
    test["before_EMPLOYED"] = test["DAYS_BIRTH"] - test["DAYS_EMPLOYED"]
    test["before_EMPLOYED_month"] = np.floor(test["before_EMPLOYED"] / 30) - (
        (np.floor(test["before_EMPLOYED"] / 30) / 12).astype(int) * 12
    )
    test["before_EMPLOYED_month"] = test["before_EMPLOYED_month"].astype(int)
    test["before_EMPLOYED_week"] = np.floor(test["before_EMPLOYED"] / 7) - (
        (np.floor(test["before_EMPLOYED"] / 7) / 4).astype(int) * 4
    )
    test["before_EMPLOYED_week"] = test["before_EMPLOYED_week"].astype(int)

    # gender_car_reality
    train["user_code"] = (
        train["gender"].astype(str)
        + "_"
        + train["car"].astype(str)
        + "_"
        + train["reality"].astype(str)
    )
    test["user_code"] = (
        test["gender"].astype(str)
        + "_"
        + test["car"].astype(str)
        + "_"
        + test["reality"].astype(str)
    )

    del_cols = [
        "gender",
        "car",
        "reality",
        "email",
        "child_num",
        "DAYS_BIRTH",
        "DAYS_EMPLOYED",
    ]
    train.drop(train.loc[train["family_size"] > 7, "family_size"].index, inplace=True)
    train.drop(del_cols, axis=1, inplace=True)
    test.drop(del_cols, axis=1, inplace=True)

    cat_cols = [
        "income_type",
        "edu_type",
        "family_type",
        "house_type",
        "occyp_type",
        "user_code",
    ]

    for col in cat_cols:
        label_encoder = LabelEncoder()
        label_encoder = label_encoder.fit(train[col])
        train[col] = label_encoder.transform(train[col])
        test[col] = label_encoder.transform(test[col])

    return train, test

# Model

In [4]:
# Catboost
def stratified_kfold_cat(
    params: Dict[str, Union[int, float, str, List[str]]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    cat_oof = np.zeros((X.shape[0], 3))
    cat_preds = np.zeros((X_test.shape[0], 3))
    cat_cols = [c for c in X.columns if X[c].dtypes == "int64"]

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
        valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

        model = CatBoostClassifier(**params)

        model.fit(
            train_data,
            eval_set=valid_data,
            early_stopping_rounds=100,
            use_best_model=True,
            verbose=100,
        )

        cat_oof[valid_idx] = model.predict_proba(X_valid)
        cat_preds += model.predict_proba(X_test) / n_fold

    log_score = log_loss(y, cat_oof)
    print(f"Log Loss Score: {log_score:.5f}\n")
    return cat_oof, cat_preds


# Light GBM
def stratified_kfold_lgbm(
    params: Dict[str, Union[int, float, str]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    lgb_oof = np.zeros((X.shape[0], 3))
    lgb_preds = np.zeros((X_test.shape[0], 3))

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        pre_model = LGBMClassifier(**params)

        pre_model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=100,
        )
        params2 = params.copy()
        params2["learning_rate"] = params["learning_rate"] * 0.1

        model = LGBMClassifier(**params2)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=100,
            init_model=pre_model,
        )
        lgb_oof[valid_idx] = model.predict_proba(X_valid)
        lgb_preds += model.predict_proba(X_test) / n_fold

    log_score = log_loss(y, lgb_oof)
    print(f"Log Loss Score: {log_score:.5f}")

    return lgb_oof, lgb_preds


# XGB
def stratified_kfold_xgb(
    params: Dict[str, Union[int, float, str]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:

    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    xgb_oof = np.zeros((X.shape[0], 3))
    xgb_preds = np.zeros((X_test.shape[0], 3))

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        model = XGBClassifier(**params)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=100,
        )

        xgb_oof[valid_idx] = model.predict_proba(X_valid)
        xgb_preds += model.predict_proba(X_test) / n_fold

    log_score = log_loss(y, xgb_oof)
    print(f"Log Loss Score: {log_score:.5f}")

    return xgb_oof, xgb_preds


# Random Foreset
def stratified_kfold_rf(
    params: Dict[str, Union[int, float, str, bool]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:

    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    rf_oof = np.zeros((X.shape[0], 3))
    rf_preds = np.zeros((X_test.shape[0], 3))

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        model = RandomForestClassifier(**params)
        model.fit(
            X_train,
            y_train,
        )

        rf_oof[valid_idx] = model.predict_proba(X_valid)
        rf_preds += model.predict_proba(X_test) / n_fold
        print(f"Log Loss Score: {log_loss(y_valid, rf_oof[valid_idx]):.5f}")

    log_score = log_loss(y, rf_oof)
    print(f"Log Loss Score: {log_score:.5f}")

    return rf_oof, rf_preds

# Cat Train
+ catboost 같은 경우 categorical한 feature를 고정시켜주는게 핵심이라고 생각하여 하이퍼파라미터튜닝에 신경을 씀
+ 하이퍼파라미터튜닝은 optuna 라이브러리로 했음

In [5]:
train_cat, test_cat = cat_load_dataset()
X = train_cat.drop("credit", axis=1)
y = train_cat["credit"]

X_test = test_cat.copy()

In [6]:
cat_params = {
    "learning_rate": 0.026612467217016746,
    "l2_leaf_reg": 0.3753065117824262,
    "max_depth": 8,
    "bagging_temperature": 1,
    "min_data_in_leaf": 57,
    "max_bin": 494,
    "random_state": 42,
    "eval_metric": "MultiClass",
    "loss_function": "MultiClass",
    "od_type": "Iter",
    "od_wait": 500,
    "iterations": 10000,
    "cat_features": [
        "income_total",
        "income_type",
        "edu_type",
        "family_type",
        "house_type",
        "FLAG_MOBIL",
        "work_phone",
        "phone",
        "occyp_type",
        "begin_month",
        "DAYS_BIRTH_month",
        "DAYS_BIRTH_week",
        "Age",
        "DAYS_EMPLOYED_month",
        "DAYS_EMPLOYED_week",
        "before_EMPLOYED",
        "before_EMPLOYED_month",
        "before_EMPLOYED_week",
        "user_code",
    ],
}

cat_oof, cat_preds = stratified_kfold_cat(cat_params, 10, X, y, X_test)

============ Fold 0 ============

0:	learn: 1.0832640	test: 1.0829576	best: 1.0829576 (0)	total: 301ms	remaining: 50m 14s
100:	learn: 0.7323907	test: 0.6929501	best: 0.6929501 (100)	total: 37.1s	remaining: 1h 37s
200:	learn: 0.7058060	test: 0.6735975	best: 0.6735975 (200)	total: 1m 27s	remaining: 1h 10m 51s
300:	learn: 0.6885144	test: 0.6706689	best: 0.6706689 (300)	total: 2m 22s	remaining: 1h 16m 18s
400:	learn: 0.6701975	test: 0.6682080	best: 0.6681988 (388)	total: 3m 16s	remaining: 1h 18m 16s
500:	learn: 0.6494751	test: 0.6664689	best: 0.6662605 (486)	total: 4m 11s	remaining: 1h 19m 23s
600:	learn: 0.6290599	test: 0.6657848	best: 0.6657848 (600)	total: 5m 6s	remaining: 1h 19m 56s
700:	learn: 0.6080674	test: 0.6648291	best: 0.6648201 (698)	total: 6m 2s	remaining: 1h 20m 7s
800:	learn: 0.5895904	test: 0.6646412	best: 0.6643652 (761)	total: 6m 58s	remaining: 1h 20m
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6643651535
bestIteration = 761

Shrink model to first

# LGBM Train
+ 하이퍼파라미터 튜닝은 optuna를 사용했음

In [7]:
train, test = load_dataset()
X = train.drop("credit", axis=1)
y = train["credit"]
X_test = test.copy()

In [8]:
lgb_params = {
    "reg_alpha": 5.998770177220496e-05,
    "reg_lambda": 0.07127674208132959,
    "max_depth": 18,
    "num_leaves": 125,
    "colsample_bytree": 0.4241631237880101,
    "subsample": 0.8876057928391585,
    "subsample_freq": 5,
    "min_child_samples": 5,
    "max_bin": 449,
    "random_state": 42,
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "n_estimators": 10000,
    "objective": "multiclass",
    "metric": "multi_logloss",
}
lgbm_oof, lgbm_preds = stratified_kfold_lgbm(lgb_params, 10, X, y, X_test)

============ Fold 0 ============

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.558297	valid_1's multi_logloss: 0.710048
[200]	training's multi_logloss: 0.444307	valid_1's multi_logloss: 0.68103
[300]	training's multi_logloss: 0.370455	valid_1's multi_logloss: 0.675189
[400]	training's multi_logloss: 0.314399	valid_1's multi_logloss: 0.680516
Early stopping, best iteration is:
[311]	training's multi_logloss: 0.364184	valid_1's multi_logloss: 0.674869
Training until validation scores don't improve for 100 rounds
[400]	training's multi_logloss: 0.358725	valid_1's multi_logloss: 0.674764
[500]	training's multi_logloss: 0.352808	valid_1's multi_logloss: 0.67485
Early stopping, best iteration is:
[407]	training's multi_logloss: 0.358272	valid_1's multi_logloss: 0.674724
============ Fold 1 ============

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 0.557051	valid_1's multi_logloss: 0.715119
[20

# XGB Train
+ 하이퍼파라미터 튜닝을 optuna를 사용했음

In [9]:
xgb_params = {
    "eta": 0.023839252347297356,
    "reg_alpha": 6.99554614267605e-06,
    "reg_lambda": 0.010419988953061583,
    "max_depth": 15,
    "max_leaves": 159,
    "colsample_bytree": 0.4515469593932409,
    "subsample": 0.7732694309118915,
    "min_child_weight": 5,
    "gamma": 0.6847131315687576,
    "random_state": 42,
    "n_estimators": 10000,
    "objective": "multi:softmax",
    "eval_metric": "mlogloss",
}
xgb_oof, xgb_preds = stratified_kfold_xgb(xgb_params, 10, X, y, X_test)

============ Fold 0 ============

[0]	validation_0-mlogloss:1.08671	validation_1-mlogloss:1.08830


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[100]	validation_0-mlogloss:0.60860	validation_1-mlogloss:0.73678
[200]	validation_0-mlogloss:0.49016	validation_1-mlogloss:0.69367
[300]	validation_0-mlogloss:0.42604	validation_1-mlogloss:0.68027
[400]	validation_0-mlogloss:0.38186	validation_1-mlogloss:0.67709
[497]	validation_0-mlogloss:0.34856	validation_1-mlogloss:0.67779


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


============ Fold 1 ============

[0]	validation_0-mlogloss:1.08610	validation_1-mlogloss:1.08795
[100]	validation_0-mlogloss:0.61238	validation_1-mlogloss:0.74311
[200]	validation_0-mlogloss:0.49265	validation_1-mlogloss:0.70016
[300]	validation_0-mlogloss:0.42657	validation_1-mlogloss:0.68767
[400]	validation_0-mlogloss:0.38175	validation_1-mlogloss:0.68516
[500]	validation_0-mlogloss:0.34796	validation_1-mlogloss:0.68706
[518]	validation_0-mlogloss:0.34252	validation_1-mlogloss:0.68770
============ Fold 2 ============

[0]	validation_0-mlogloss:1.08659	validation_1-mlogloss:1.08834
[100]	validation_0-mlogloss:0.61568	validation_1-mlogloss:0.74170
[200]	validation_0-mlogloss:0.49399	validation_1-mlogloss:0.69688
[300]	validation_0-mlogloss:0.42702	validation_1-mlogloss:0.68586
[400]	validation_0-mlogloss:0.38326	validation_1-mlogloss:0.68467
[466]	validation_0-mlogloss:0.35951	validation_1-mlogloss:0.68507
============ Fold 3 ============

[0]	validation_0-mlogloss:1.08586	validation

# RandomForest Train

In [10]:
rf_params = {
        "criterion": "gini",
        "n_estimators": 300,
        "min_samples_split": 10,
        "min_samples_leaf": 2,
        "max_features": "auto",
        "oob_score": True,
        "random_state": 42,
        "n_jobs": -1,
    }
rf_oof, rf_preds = stratified_kfold_rf(rf_params, 10, X, y, X_test)

============ Fold 0 ============

Log Loss Score: 0.68083
============ Fold 1 ============

Log Loss Score: 0.68767
============ Fold 2 ============

Log Loss Score: 0.68559
============ Fold 3 ============

Log Loss Score: 0.69447
============ Fold 4 ============

Log Loss Score: 0.68046
============ Fold 5 ============

Log Loss Score: 0.70892
============ Fold 6 ============

Log Loss Score: 0.69037
============ Fold 7 ============

Log Loss Score: 0.69148
============ Fold 8 ============

Log Loss Score: 0.69465
============ Fold 9 ============

Log Loss Score: 0.69926
Log Loss Score: 0.69137


In [11]:
train, test = load_dataset()
train_x = train.drop("credit", axis = 1)
train_y = train['credit'].values

In [12]:
train_pred = np.concatenate([cat_oof, lgbm_oof, xgb_oof, rf_oof], axis=1)
train_pred.shape

(26451, 12)

In [13]:
test_pred = np.concatenate([cat_preds, lgbm_preds, xgb_preds, rf_preds], axis=1)
test_pred.shape

(10000, 12)

# Pytorch Tabular
+ Stacking Ensemble을 사용하며 학습 진행

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
n_fold = 10
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
splits = folds.split(train_pred, train_y)
net_oof = np.zeros((train_pred.shape[0], 3))
net_preds = np.zeros((test_pred.shape[0], 3))
for fold, (train_idx, valid_idx) in enumerate(splits):
    print(f"============ Fold {fold} ============\n")
    X_train, X_valid = train_pred[train_idx], train_pred[valid_idx]
    y_train, y_valid = train_y[train_idx], train_y[valid_idx]
    model = TabNetMultiTaskClassifier(
            n_d=64, n_a=64, n_steps=1,
            lambda_sparse=1e-4,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=2e-2),
            scheduler_params = {"gamma": 0.9, "step_size": 50},
            scheduler_fn=torch.optim.lr_scheduler.StepLR,
            mask_type="entmax", 
            device_name=device
    )

    model.fit(
        X_train, y_train.reshape(-1,1),
        eval_set=[(X_valid, y_valid.reshape(-1,1))],
        max_epochs=100,
        batch_size=1024,
        eval_metric=["logloss"],
        virtual_batch_size=128,
        num_workers=1,
        drop_last=False
    )
    net_oof[valid_idx] = model.predict_proba(X_valid)
    net_preds += model.predict_proba(test_pred)[0] / n_fold
log_score = log_loss(train_y, net_oof)
print(f"Log Loss Score: {log_score:.5f}")

============ Fold 0 ============

Device used : cuda
epoch 0  | loss: 0.75703 | val_0_logloss: 0.79897 |  0:00:01s
epoch 1  | loss: 0.68556 | val_0_logloss: 0.77978 |  0:00:02s
epoch 2  | loss: 0.68048 | val_0_logloss: 0.75859 |  0:00:04s
epoch 3  | loss: 0.67863 | val_0_logloss: 0.75964 |  0:00:05s
epoch 4  | loss: 0.68124 | val_0_logloss: 0.75431 |  0:00:06s
epoch 5  | loss: 0.67689 | val_0_logloss: 0.73965 |  0:00:07s
epoch 6  | loss: 0.67523 | val_0_logloss: 0.7408  |  0:00:08s
epoch 7  | loss: 0.67533 | val_0_logloss: 0.7257  |  0:00:10s
epoch 8  | loss: 0.67628 | val_0_logloss: 0.7194  |  0:00:11s
epoch 9  | loss: 0.67497 | val_0_logloss: 0.70544 |  0:00:12s
epoch 10 | loss: 0.67502 | val_0_logloss: 0.71008 |  0:00:13s
epoch 11 | loss: 0.67271 | val_0_logloss: 0.68738 |  0:00:14s
epoch 12 | loss: 0.67292 | val_0_logloss: 0.68595 |  0:00:15s
epoch 13 | loss: 0.67197 | val_0_logloss: 0.68977 |  0:00:17s
epoch 14 | loss: 0.67286 | val_0_logloss: 0.66919 |  0:00:18s
epoch 15 | loss: 

In [16]:
submission = pd.read_csv("/kaggle/input/predictcreditcarddelinquency/sample_submission.csv")
submission.iloc[:, 1:] = net_preds
submission.to_csv("meta_ensemble_submit.csv", index=False)